In [71]:
import pandas as pd
import numpy as np 
import re
import math

- *a/*b
- *a+*b/*c
- *a/*b+*c
- *a+*b/*c+*d
- *a/*b+*c+*d
- *ax2/*b
- *ax2/*bx2'
- *a/*bx2
- *ax2/*b+*c

new:
- consider format *4.001

In [72]:
# diplotype > activity socre
def CYP2D6_activity_score(diplotype):
    if pd.isnull(diplotype):
        return float("NaN")
    
    activity_score = 0
    list_hap = diplotype.split('/')    #  *1/*1 ->  [*1, *1]
    
    ## path of the scoring files, get correct scoreing files
    activity_score_df = pd.read_csv('/home/ec2-user/works/data/gene-scores/CYP2D6-scores.tsv', sep = '\t') 
    
    for i in list_hap:
        list_star =   i.split('+')             # *1+*2+*3 ->   [*1, *2, *3]
        
        for j in list_star:
            if 'x' in list_star:
            
                hap = j.split('x')[0]      #   *1x2 ->   *1
                if '.' in hap:
                    hap = hap.split('.')[0]     #*4.01 -> *4
                hap = '*'+ re.findall("\d+", hap)[0]   # *4b -> *4 (remove subtypes)
                copy = j.split('x')[1]     #   *1x2 ->   2
            
                activity_score += activity_score_df[activity_score_df[activity_score_df.columns[0]] == hap]['Activity score'].values[0] * copy
                
            else:
                hap = j.split('x')[0]
                hap = '*'+ re.findall("\d+", hap)[0]    # *4b -> *4 (remove subtypes)
                activity_score += activity_score_df[activity_score_df[activity_score_df.columns[0]] == hap]['Activity score'].values[0]
        
        
    
    
    return activity_score
            
        



In [73]:
# activity score -> phenotype

def CYP2D6_phenotype(activity_score):
    if math.isnan(activity_score):
        phenotype = 'Uncertain metabolizer'
    elif activity_score >= 2.5:
        phenotype = 'Ultrarapid metabolizer'
    elif  (activity_score < 0):
        phenotype = 'Uncertain metabolizer'
    elif ((activity_score <= 2.25) and (activity_score > 1)):
        phenotype = 'Normal metabolizer'        
    elif (activity_score <= 1) and (activity_score >= 0.25):
        phenotype = 'Intermediate metabolizer'
    elif ((activity_score < 0.25) and (activity_score >= 0)):
        phenotype = 'Poor metabolizer'
        

    return phenotype

In [74]:
# Test - please input the diplotype file here
test_df = pd.read_csv('/home/ec2-user/Vanessa/test_file_2d6_diplotype_phenotype.csv')

test_df

,sample_id,cyrius_filter,cyrius_diplotype_original,cyrius_diplotype,aldy_filter,aldy_diplotype_original,aldy_diplotype,stellarpgx_filter,stellarpgx_diplotype_original,stellarpgx_diplotype,consensus_filter,consensus_diplotype
0,NA12878,PASS,*3/*68+*4,*3/*4+*68,NaN,*3/*68+*4.021,*3/*4+*68,NaN,*3/*68+*4,*3/*4+*68,PASS,*3/*4+*68
1,NA12878-2,PASS,*3/*68+*4,*3/*4+*68,NaN,*3/*68,*3/*68,NaN,*3/*68+*4,*3/*4+*68,PASS,*3/*4+*68
2,NA12878-3,PASS,*3/*68+*5,*3/*68+*4,NaN,*3/*68,*3/*68,NaN,*3/*4,*3/*4,FAIL,NaN
3,NA12878-4,PASS,*1/*1,*1/*1,NaN,*1/*13,*1/*13,NaN,*1/*22,*1/*22,FAIL,NaN
4,NA12878-3,PASS,*1/*1,*1/*1,NaN,*1/*10,*1/*10,NaN,*3/*4,*3/*4,FAIL,NaN


In [75]:
# assign activity scores for diplotype callers
test_df['cyrius_activity_score'] = test_df['cyrius_diplotype'].apply(lambda x: CYP2D6_activity_score(x))
test_df['aldy_activity_score'] = test_df['aldy_diplotype'].apply(lambda x: CYP2D6_activity_score(x))
test_df['stellarpgx_activity_score'] = test_df['stellarpgx_diplotype'].apply(lambda x: CYP2D6_activity_score(x))
test_df['consensus_activity_score'] = test_df['consensus_diplotype'].apply(lambda x: CYP2D6_activity_score(x))    
test_df

,sample_id,cyrius_filter,cyrius_diplotype_original,cyrius_diplotype,aldy_filter,aldy_diplotype_original,aldy_diplotype,stellarpgx_filter,stellarpgx_diplotype_original,stellarpgx_diplotype,consensus_filter,consensus_diplotype,cyrius_activity_score,aldy_activity_score,stellarpgx_activity_score,consensus_activity_score
0,NA12878,PASS,*3/*68+*4,*3/*4+*68,NaN,*3/*68+*4.021,*3/*4+*68,NaN,*3/*68+*4,*3/*4+*68,PASS,*3/*4+*68,0.0,0.00,0.0,0.0
1,NA12878-2,PASS,*3/*68+*4,*3/*4+*68,NaN,*3/*68,*3/*68,NaN,*3/*68+*4,*3/*4+*68,PASS,*3/*4+*68,0.0,0.00,0.0,0.0
2,NA12878-3,PASS,*3/*68+*5,*3/*68+*4,NaN,*3/*68,*3/*68,NaN,*3/*4,*3/*4,FAIL,NaN,0.0,0.00,0.0,NaN
3,NA12878-4,PASS,*1/*1,*1/*1,NaN,*1/*13,*1/*13,NaN,*1/*22,*1/*22,FAIL,NaN,2.0,1.00,NaN,NaN
4,NA12878-3,PASS,*1/*1,*1/*1,NaN,*1/*10,*1/*10,NaN,*3/*4,*3/*4,FAIL,NaN,2.0,1.25,0.0,NaN


In [76]:
# assign phenotypes for diplotype callers
test_df['cyrius_phenotype'] = test_df['cyrius_activity_score'].apply(lambda x: CYP2D6_phenotype(x))
test_df['aldy_phenotype'] = test_df['aldy_activity_score'].apply(lambda x: CYP2D6_phenotype(x))
test_df['stellarpgx_phenotype'] = test_df['stellarpgx_activity_score'].apply(lambda x: CYP2D6_phenotype(x))

test_df['consensus_phenotype'] = test_df['consensus_activity_score'].apply(lambda x: CYP2D6_phenotype(x))


In [77]:
test_df

,sample_id,cyrius_filter,cyrius_diplotype_original,cyrius_diplotype,aldy_filter,aldy_diplotype_original,aldy_diplotype,stellarpgx_filter,stellarpgx_diplotype_original,stellarpgx_diplotype,consensus_filter,consensus_diplotype,cyrius_activity_score,aldy_activity_score,stellarpgx_activity_score,consensus_activity_score,cyrius_phenotype,aldy_phenotype,stellarpgx_phenotype,consensus_phenotype
0,NA12878,PASS,*3/*68+*4,*3/*4+*68,NaN,*3/*68+*4.021,*3/*4+*68,NaN,*3/*68+*4,*3/*4+*68,PASS,*3/*4+*68,0.0,0.00,0.0,0.0,Poor metabolizer,Poor metabolizer,Poor metabolizer,Poor metabolizer
1,NA12878-2,PASS,*3/*68+*4,*3/*4+*68,NaN,*3/*68,*3/*68,NaN,*3/*68+*4,*3/*4+*68,PASS,*3/*4+*68,0.0,0.00,0.0,0.0,Poor metabolizer,Poor metabolizer,Poor metabolizer,Poor metabolizer
2,NA12878-3,PASS,*3/*68+*5,*3/*68+*4,NaN,*3/*68,*3/*68,NaN,*3/*4,*3/*4,FAIL,NaN,0.0,0.00,0.0,NaN,Poor metabolizer,Poor metabolizer,Poor metabolizer,Uncertain metabolizer
3,NA12878-4,PASS,*1/*1,*1/*1,NaN,*1/*13,*1/*13,NaN,*1/*22,*1/*22,FAIL,NaN,2.0,1.00,NaN,NaN,Normal metabolizer,Intermediate metabolizer,Uncertain metabolizer,Uncertain metabolizer
4,NA12878-3,PASS,*1/*1,*1/*1,NaN,*1/*10,*1/*10,NaN,*3/*4,*3/*4,FAIL,NaN,2.0,1.25,0.0,NaN,Normal metabolizer,Normal metabolizer,Poor metabolizer,Uncertain metabolizer


Have a feature to check consensus across diplotype callers. 
if consensus_filter is FAIL, return majority results across the callers. 

Examples: 

cyrius_phenotype	    aldy_phenotype	    stellarpgx_phenotype	consensus_phenotype

Normal metabolizer     Normal metabolizer    Normal metabolizer ---> Normal metabolizer
Poor metabolizer       Normal metabolizer    Poor metabolizer   ---> Poor metabolizer
Normal metabolizer     Poor metabolizer      Uncertain metabolizer ---> NA 

In [78]:
from collections import Counter

def getMajority(row): 
    #phenotype determined previously 
    consensus_phenotype = row['consensus_phenotype']
    
    #return majority only if consensus FAIL
    if row['consensus_filter'] != 'PASS':   
        calls = []
        
        #get metabolizer type from all 3 callers 
        cyrius_call = str(row['cyrius_phenotype'])
        aldy_call = str(row['aldy_phenotype'])
        stellar_call = str(row['stellarpgx_phenotype'])
        
        #if 3 callers are all different, return NA 
        if cyrius_call != aldy_call and aldy_call != stellar_call and cyrius_call != stellar_call: 
            return 'NA' 
        
        #get metabolizer type, i.e 'Poor Metabolizer' -> 'Poor'
        calls.append(cyrius_call.split()[0])
        calls.append(aldy_call.split()[0])
        calls.append(stellar_call.split()[0])
        
        #use counter to find the majority 
        counts = Counter(calls)
        
        #get the top common metabolizer type, i.e [('Poor': 3)]
        majority = counts.most_common(1)[0][0]
        
        #reattach 'metabolizer', i.e  'Poor' --> 'Poor metabolizer'
        majority += ' metabolizer'
        return majority
    else: 
        return consensus_phenotype

In [79]:
test_df['consensus_phenotype'] = test_df.apply(lambda x: getMajority(x), axis=1)

Counter({'Poor': 3})
Counter({'Normal': 2, 'Poor': 1})


In [80]:
test_df

,sample_id,cyrius_filter,cyrius_diplotype_original,cyrius_diplotype,aldy_filter,aldy_diplotype_original,aldy_diplotype,stellarpgx_filter,stellarpgx_diplotype_original,stellarpgx_diplotype,consensus_filter,consensus_diplotype,cyrius_activity_score,aldy_activity_score,stellarpgx_activity_score,consensus_activity_score,cyrius_phenotype,aldy_phenotype,stellarpgx_phenotype,consensus_phenotype
0,NA12878,PASS,*3/*68+*4,*3/*4+*68,NaN,*3/*68+*4.021,*3/*4+*68,NaN,*3/*68+*4,*3/*4+*68,PASS,*3/*4+*68,0.0,0.00,0.0,0.0,Poor metabolizer,Poor metabolizer,Poor metabolizer,Poor metabolizer
1,NA12878-2,PASS,*3/*68+*4,*3/*4+*68,NaN,*3/*68,*3/*68,NaN,*3/*68+*4,*3/*4+*68,PASS,*3/*4+*68,0.0,0.00,0.0,0.0,Poor metabolizer,Poor metabolizer,Poor metabolizer,Poor metabolizer
2,NA12878-3,PASS,*3/*68+*5,*3/*68+*4,NaN,*3/*68,*3/*68,NaN,*3/*4,*3/*4,FAIL,NaN,0.0,0.00,0.0,NaN,Poor metabolizer,Poor metabolizer,Poor metabolizer,Poor metabolizer
3,NA12878-4,PASS,*1/*1,*1/*1,NaN,*1/*13,*1/*13,NaN,*1/*22,*1/*22,FAIL,NaN,2.0,1.00,NaN,NaN,Normal metabolizer,Intermediate metabolizer,Uncertain metabolizer,NA
4,NA12878-3,PASS,*1/*1,*1/*1,NaN,*1/*10,*1/*10,NaN,*3/*4,*3/*4,FAIL,NaN,2.0,1.25,0.0,NaN,Normal metabolizer,Normal metabolizer,Poor metabolizer,Normal metabolizer
